# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import numpy and pandas

import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [3]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [11]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


In [4]:
# Your code here:

salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
# Your code here:
salaries.isna().sum() 

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [8]:
# Your code here:
salaries['Salary or Hourly'].value_counts().Hourly

8022

In [9]:
salaries['Salary or Hourly'].value_counts().Salary

25161

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [10]:
salaries.Department.value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [39]:


# Your code here:
import seaborn as sns
import scipy.stats as stats
import numpy as np
import random
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import binomtest
from scipy.stats import binom
from scipy.stats import ttest_ind, norm, t
from statsmodels.stats.weightstats import ztest as ztest

In [41]:
hourly_wage = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate'].dropna()

z_stat, p_value = ztest(hourly_wage, value=30)
p_value

1.8228873859286195e-94

In [ ]:
# expected p-value 👇🏾
# 1.8228873859286195e-94

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [43]:
police_salaries = salaries[(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')]['Annual Salary']
test_statistic, p_value = ttest_1samp(police_salaries, 86000, alternative='greater')
p_value

0.0010301701775482569

In [44]:
test_statistic

3.081997005712994

In [ ]:
# expected p-value 👇🏾
# 0.0010280845287547855

Using the `crosstab` function, find the department that has the most hourly workers. 

In [46]:
hourly_workers = salaries.loc[salaries['Salary or Hourly'] == 'Hourly']
hourly_by_department = pd.crosstab(hourly_workers.Department, hourly_workers['Salary or Hourly'])

hourly_by_department = hourly_by_department.sort_values('Hourly', ascending=False)

hourly_by_department.index[0]

'STREETS & SAN'

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [49]:
hourly_ss = salaries[(salaries['Salary or Hourly'] == 'Hourly') & (salaries['Department'] == 'STREETS & SAN')]

t_stat, p_value = ttest_1samp(hourly_ss['Hourly Rate'], 35, alternative='less')

print(p_value)

1.6689265282353859e-21


In [ ]:
# expected p-value 👇🏾
# 1.6689265282353859e-21

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [53]:
from statsmodels.stats.proportion import proportions_ztest

n_hourly = len(salaries.loc[salaries['Salary or Hourly'] == 'Hourly'])

n_total = len(salaries)

null_proportion = 0.25

stat, pval = proportions_ztest(n_hourly, n_total, null_proportion)
pval

0.0004481127249057967

In [ ]:
# expected p-value 👇🏾
# 0.0004481127249057967

# BONUS: Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [59]:
hourly_wage = salaries.loc[salaries['Salary or Hourly'] == 'Hourly', 'Hourly Rate']
hourly_mean = hourly_wage.mean()
hourly_std_error = stats.sem(hourly_wage)

confidence_level = 0.95
degrees_freedom = len(hourly_wage) - 1
lower, upper = stats.t.interval(confidence_level, degrees_freedom, hourly_mean, hourly_std_error)

({lower}, {upper})

({32.52345834488425}, {33.05365708767623})

In [ ]:
# expected C-I 👇🏾
# (32.52345834488529, 33.05365708767727)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [63]:
police_salaries = salaries[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary')]['Annual Salary']

sample_mean = police_salaries.mean()
sample_std = police_salaries.std(ddof=1)
n = len(police_salaries)

conf_level = 0.95

conf_int = stats.t.interval(conf_level, n - 1, loc=sample_mean, scale=sample_std / (n ** 0.5))

{conf_int}


{(86177.05631531784, 86795.77269094894)}

In [ ]:
# expected C-I 👇🏾
# (86177.05631531784, 86795.77269094894)